### LSTM with Stocashtic Gradient Descent 

In [1]:
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
from tqdm import tqdm # Used to display training progress bar
import numpy as np
from scipy import misc
import time

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes])),
    'last': tf.Variable(tf.random_normal([n_hidden, n_classes])),
    'lay_27': tf.Variable(tf.random_normal([n_hidden, n_classes])),
    'lay_26': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes])),
    'last': tf.Variable(tf.random_normal([n_classes])),
    'lay_27':tf.Variable(tf.random_normal([n_classes])),
    'lay_26': tf.Variable(tf.random_normal([n_classes]))
}


In [10]:
x = tf.unstack(x, n_steps, 1)
# Define a lstm cell with tensorflow
lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, reuse=True)

with tf.variable_scope("myrnn") as scope:
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    outputs2, states2 = rnn.static_rnn(lstm_cell, x[0:27], dtype=tf.float32)
    outputs3, states3 = rnn.static_rnn(lstm_cell, x[0:26], dtype=tf.float32)
    outputs1, states1 = rnn.static_rnn(lstm_cell, x[0:14], dtype=tf.float32)

pred=tf.matmul(outputs[-1], weights['out']) + biases['out']
pred_last=tf.matmul(states[-1], weights['last']) + biases['last']
pred_27=tf.matmul(states2[-1], weights['lay_27']) + biases['lay_27']
pred_26=tf.matmul(states3[-1], weights['lay_26']) + biases['lay_26'] 

In [13]:
def train_layer_sg(l_t, l_sg, h_n):
    grad_true = tf.gradients(l_t, h_n)
    grad_pred = tf.gradients(l_sg, h_n)
    print(type(l_t))
    print(type(l_sg))
    print(type(grad_pred))
    print(grad_pred)
    print(len(grad_pred))
    print(type(grad_true))
    sg_loss = tf.add (tf.reduce_sum(tf.abs( tf.subtract(l_t, l_sg))) , tf.reduce_sum(tf.abs( tf.subtract(grad_pred , grad_true))))
    sg_opt = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(sg_loss)
    return sg_opt, sg_loss

In [14]:
pred_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
pred_loss_last=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_last, labels=y))
pred_loss_sg1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_27, labels=y))
pred_loss_sg2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_26, labels=y))

        #                     train_layer_n(n, h_m, h_n,     d_hat_m, class_loss, d_n=None):
layer_block4_opt = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum = 0.9).minimize(pred_loss_last)
        
sg3_opt, sg3_loss = train_layer_sg(pred_loss_last, pred_loss_sg1, states2[-1])
        
layer_block3_opt = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum = 0.9).minimize(states2[-1], 
grad_loss= tf.cast(tf.gradients(pred_loss_last,states[-1]), tf.float32)[0])

sg2_opt, sg2_loss = train_layer_sg(pred_loss_last, pred_loss_sg2, states3[-1])
        
layer_block2_opt = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum = 0.9).minimize(states3[-1], 
            grad_loss= tf.cast(tf.gradients(pred_loss_sg2, states2[-1]), tf.float32)[0])


backprop_opt = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum = 0.9).minimize(pred_loss)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'list'>
[<tf.Tensor 'gradients_5/MatMul_10_grad/MatMul:0' shape=(?, 128) dtype=float32>]
1
<class 'list'>


TypeError: Expected float32 passed to parameter 'y' of op 'Sub', got [None] of type 'list' instead.

In [38]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [4]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(backprop_opt, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(pred_loss, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

In [ ]:
with sg_sess.as_default():
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Each layer can now be independently updated (could be parallelized)
        sg_sess.run([layer_block2_opt], feed_dict={X:batch_x,Y:batch_y})    
        sg_sess.run([layer_block3_opt], feed_dict={X:batch_x,Y:batch_y})
        sg_sess.run([layer_block4_opt], feed_dict={X:batch_x,Y:batch_y})

        sg_sess.run([sg2_opt], feed_dict={X:batch_x,Y:batch_y})
        sg_sess.run([sg3_opt], feed_dict={X:batch_x,Y:batch_y})
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(pred_loss, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
    batch_accuracy, batch_loss = sg_sess.run([accuracy, pred_loss], feed_dict={X:val_X,Y:val_y})